<a href="https://colab.research.google.com/github/Akshaykumarmundrathi/GCN/blob/main/GCN_Task3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified GCN, according to Task 1.1:

In [1]:
!unzip GCN_export.zip

Archive:  GCN_export.zip
   creating: data/
   creating: data/cora/
  inflating: data/cora/cora.cites    
  inflating: data/cora/cora.content  
  inflating: data/cora/README        


In [2]:
import numpy as np
import scipy.sparse as sp
import torch

In [3]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [4]:
def feature_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [5]:
def adj_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1)) # Sum each row
    r_inv = np.power(rowsum, -1/2).flatten() # Negative square root
#     r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv) # Create diagonal matrix

    # D^(-1/2).A.D^(-1/2)
    mx = r_mat_inv.dot(mx)
    mx = mx.dot(r_mat_inv)
    return mx

In [6]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [7]:
def limit_neighbors(adj_csr, max_neighbors=10, random_seed=42):
    """
    Limit the number of neighbors for each node in the adjacency matrix.

    Parameters:
    adj_csr (sp.csr_matrix): The original adjacency matrix in CSR format.
    max_neighbors (int): The maximum number of neighbors per node.

    Returns:
    sp.csr_matrix: The modified adjacency matrix with limited neighbors in CSR format.
    """
    adj_coo = adj_csr.tocoo()  # Convert to COO format

    if max_neighbors >= adj_coo.shape[0]:
        return adj_csr

    random_state = np.random.RandomState(random_seed)  # Ensures reproducibility

    rows = adj_coo.row
    cols = adj_coo.col
    data = adj_coo.data

    new_rows = []
    new_cols = []
    new_data = []

    for i in range(adj_coo.shape[0]):
        mask = rows == i
        neighbors = cols[mask]
        if len(neighbors) > max_neighbors:
            neighbors = random_state.choice(neighbors, max_neighbors, replace=False)
        new_rows.extend([i] * len(neighbors))
        new_cols.extend(neighbors)
        new_data.extend([1] * len(neighbors))  # Assuming unweighted graph

    new_adj_coo = sp.coo_matrix((new_data, (new_rows, new_cols)), shape=adj_coo.shape)
    return new_adj_coo.tocsr()  # Convert back to CSR format


In [8]:
def load_data(path="/content/data/cora/", dataset="cora"):

    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32) # Processing features into a sparse matrix
    labels = encode_onehot(idx_features_labels[:, -1]) # one-hot encoding the labels


    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32) # Reading node-ids
    idx_map = {j: i for i, j in enumerate(idx)} # Creating index for nodes to map it in adjacency matrix

    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32) # Reading edges
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape) # Mapping node-ids in the edge list to the index

    # Build adjacency matrix
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # CHECK OUT THE DIFFERENCES BETWEEN csr_matrix (features) and coo_matrix (adj)

    # Normalizing features
    features = feature_normalize(features)

#     # build symmetric adjacency matrix
#     adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    # Normalizing the adjacency matrix after adding self loops
    adj_original = adj_normalize(adj + sp.eye(adj.shape[0]))
    """
    To ensure that each of the two hidden layers aggregates from different random samples of neighbors, you need to create two separate modified adjacency matrices,
     one for each layer, with different random neighbor selections.
Here's how you can adjust the process:

Step 1: Modify the Adjacency Matrix Twice with Different Random Seeds
    """

    # Limit the number of neighbors for the first layer
    adj_layer1 = limit_neighbors(adj_original, max_neighbors=10, random_seed=42)

    # Limit the number of neighbors for the second layer with a different random seed
    adj_layer2 = limit_neighbors(adj_original, max_neighbors=10, random_seed=24)

    adj_layer1 = sparse_mx_to_torch_sparse_tensor(adj_layer1)
    adj_layer2 = sparse_mx_to_torch_sparse_tensor(adj_layer2)



    # Setting training, validation, and test range
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    # Converting all matrices into pytorch tensors
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj_layer1, adj_layer2, features, labels, idx_train, idx_val, idx_test  #return them

In [9]:
# Function to find accuracy from two tensors
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels) # Get the index of maximum value of 1 dimension and typecast to labels datatype
    correct = preds.eq(labels).double() # Convert into double
    correct = correct.sum() # Sum correct predictions
    return correct / len(labels)

In [10]:
import math

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

# Class to define a neural network layer that inherits PyTorch Module
# Check out documentaion of the base class 'Module' at:
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class GraphConvolution(Module):
    # Each layer requires no. of input features, no. of output features, and optional bias
    def __init__(self, in_feat, out_feat, bias=True):
        super(GraphConvolution, self).__init__()

        self.in_features = in_feat
        self.out_features = out_feat

        # Using Parameter to automatically add weights and bias to learnable parameters
        #THIS WILL BE USEFUL ONLY WHEN WE USE Module in the model
        self.weight = Parameter(torch.FloatTensor(in_feat, out_feat))

        if bias:
            self.bias = Parameter(torch.FloatTensor(out_feat))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    # Function to get uniform distribution of weights and bias values
    # Can be removed if necessary
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    # Forward function where it actually requires the input data and operations
    def forward(self, inp, adj):
        # Basically we multiply A.H,W
        support = torch.mm(inp, self.weight)
        output = torch.spmm(adj, support)

        # Adding bias if true
        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [11]:
import torch.nn as nn
"""
import torch.nn.functional as F

# Class to define the model architecture
class GCN(nn.Module):
    # The model needs no. of input features, no. of hidden units,
    # no. of classes, and optional dropout

    # NOTE: We use a simply model with one hidden layer
        # Architecture will change for deep models
        # Ideally, we keep only a few layers in most GNNs
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        # Defining one hidden layer and one output layer
        self.gcn1 = GraphConvolution(nfeat, nhid)
        self.gcn2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    # Similar to GraphConvolution, we give required input data to the forward function
    # And specify operations - here it is activation and dropout
    def forward(self, x, adj):
        x = F.relu(self.gcn1(x, adj)) # Applying non-linearity on hidden layer 1
        # Checkout difference between nn.Dropout() and F.dropout()
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gcn2(x, adj)
        return F.log_softmax(x, dim=1) # Applying lograthmic softmax on output layer

"""
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nhid2, nclass, dropout):
        super(GCN, self).__init__()
        self.gcn1 = GraphConvolution(nfeat, nhid)
        self.gcn2 = GraphConvolution(nhid, nhid2)
        self.gcn3 = GraphConvolution(nhid2, nclass)
        self.dropout = dropout
#####Step 2: Modify the GCN Forward Pass
#########Adjust the forward pass of your GCN model to accept two adjacency matrices and use them for the different layers.
    def forward(self, x, adj_layer1, adj_layer2):
        x = F.relu(self.gcn1(x, adj_layer1))
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gcn2(x, adj_layer2))
        x = self.gcn3(x, adj_layer2)
        return F.log_softmax(x, dim=1)


In [12]:
import torch.optim as optim

adj_layer1, adj_layer2, features, labels, train_ids, val_ids, test_ids = load_data()
"""
# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)
"""
model = GCN(nfeat=features.shape[1],
            nhid=16,
            nhid2=16,
            nclass=labels.max().item() + 1,
            dropout=0.5)


# Using Adam optimizer. Other optimizer can be used too
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

Loading cora dataset...


/tmp/ipython-input-6-3650411315.py:8: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:644.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [13]:
# Code for GPU computing

# CHANGE THIS CODE TO SUIT YOUR VERSION OF PYTORCH. THE SYNTAX OF THIS COULD VARY SIGNIFICANTLY

# If cuda is available, move all data to gpu
# And preparing for CUDA operations
if torch.cuda.is_available():
    model.cuda()
    features = features.cuda()
    adj_layer1 = adj_layer1.cuda()
    adj_layer2 = adj_layer2.cuda()
    labels = labels.cuda()
    train_ids.cuda()
    val_ids.cuda()
    test_ids.cuda()

In [14]:
import time

def train(epoch):
    t = time.time()

    # Evaluating the model in training mode
    model.train()
    optimizer.zero_grad() # Reseting gradient at each layer to avoid exploding gradient problem
    output = model(features, adj_layer1, adj_layer2)
     # Optimizing with nll_loss. Other losses like cross_entropy can also be used
    loss_train = F.nll_loss(output[train_ids], labels[train_ids])
    acc_train = accuracy(output[train_ids], labels[train_ids])
    # backprop and optimize model parameters
    # Not needed to specify parameters when using Parameter
    loss_train.backward()
    optimizer.step()

    # Evaluating validation performance separately
    model.eval()
    output = model(features, adj_layer1, adj_layer2)
    loss_val = F.nll_loss(output[val_ids], labels[val_ids])
    acc_val = accuracy(output[val_ids], labels[val_ids])


    loss_val = F.nll_loss(output[val_ids], labels[val_ids])
    acc_val = accuracy(output[val_ids], labels[val_ids])

    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [15]:
# Function to test the model
"""
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[test_ids], labels[test_ids])
    acc_test = accuracy(output[test_ids], labels[test_ids])


    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
"""

from sklearn.metrics import precision_score, recall_score, f1_score

def test():
    model.eval()
    with torch.no_grad():
        output = model(features, adj_layer1, adj_layer2)
        loss_test = F.nll_loss(output[test_ids], labels[test_ids])
        acc_test = accuracy(output[test_ids], labels[test_ids])

        _, predicted = torch.max(output[test_ids], 1)

        # Moving the data to CPU for sklearn metrics calculation
        predicted_np = predicted.cpu().numpy()
        labels_np = labels[test_ids].cpu().numpy()

        precision = precision_score(labels_np, predicted_np, average='weighted')
        recall = recall_score(labels_np, predicted_np, average='weighted')
        f1 = f1_score(labels_np, predicted_np, average='weighted')

        print(f"Test set results: loss= {loss_test.item():.4f}, accuracy= {acc_test:.4f}, precision= {precision:.4f}, recall= {recall:.4f}, F1 Score= {f1:.4f}")


In [16]:
t_total = time.time()

for epoch in range(200):
    train(epoch)

print()
print('Optimization finished...')
print('Total time elapsed: {:.4f}s'.format(time.time() - t_total))

test()

Epoch: 0001 loss_train: 1.9729 acc_train: 0.2071 loss_val: 1.9338 acc_val: 0.2000 time: 0.1428s
Epoch: 0002 loss_train: 1.8903 acc_train: 0.1714 loss_val: 1.9384 acc_val: 0.1633 time: 0.0281s
Epoch: 0003 loss_train: 1.8436 acc_train: 0.2071 loss_val: 1.9312 acc_val: 0.1767 time: 0.0290s
Epoch: 0004 loss_train: 1.8394 acc_train: 0.2286 loss_val: 1.9031 acc_val: 0.2133 time: 0.0316s
Epoch: 0005 loss_train: 1.7863 acc_train: 0.2429 loss_val: 1.8754 acc_val: 0.2667 time: 0.0295s
Epoch: 0006 loss_train: 1.7326 acc_train: 0.3071 loss_val: 1.8526 acc_val: 0.3067 time: 0.0294s
Epoch: 0007 loss_train: 1.7058 acc_train: 0.3571 loss_val: 1.8296 acc_val: 0.3100 time: 0.0321s
Epoch: 0008 loss_train: 1.6744 acc_train: 0.3429 loss_val: 1.8022 acc_val: 0.3633 time: 0.0313s
Epoch: 0009 loss_train: 1.6659 acc_train: 0.3929 loss_val: 1.7724 acc_val: 0.4233 time: 0.0287s
Epoch: 0010 loss_train: 1.6056 acc_train: 0.4429 loss_val: 1.7432 acc_val: 0.4467 time: 0.0278s
Epoch: 0011 loss_train: 1.5701 acc_train

###